In [1]:
import os
import pandas as pd

import math
import time

from uuid import uuid4
from clickhouse_driver import Client
from dotenv import load_dotenv

In [2]:
load_dotenv()

USER = str(os.getenv('CLICKHOUSE_USER'))
PASSWORD = str(os.getenv('CLICKHOUSE_PASSWORD'))
HOST = str(os.getenv('CLICKHOUSE_HOST'))
DATABASE = str(os.getenv('CLICKHOUSE_DB'))

In [3]:
CH_CLIENT = Client(
    host=HOST,  # IP-адрес сервера ClickHouse
    user=USER,  # Имя пользователя для подключения
    password=PASSWORD,  # Пароль для подключения
    database=DATABASE # База данных, к которой подключаемся
)


In [4]:
def insert_daily_data(df, table_name, chunk_size = 100_000):

    # Преобразуем столбцы в нужные типы
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['user_id'] = df['user_id'].astype('UInt32')
    df['first_visit_unique'] = pd.to_datetime(df['first_visit_unique'])  # Преобразуем в datetime
    df['visits_per_day'] = df['visits_per_day'].astype('UInt32')  
    df['group'] = df['group'].astype('Int32')

    # Создаем уникальные UUID для каждой строки данных (дополнительный первичный ключ)
    df['id'] = [str(uuid4()) for _ in range(len(df))]

    
    
    # SQL-запрос для создания таблицы с дополнительным первичным ключом
    sql_request = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        id UUID DEFAULT generateUUIDv4(),  -- Дополнительный первичный ключ
        datetime DateTime,
        user_id UInt32,
        action String,
        type String,
        first_visit_unique DateTime,  -- Преобразуем в DateTime
        visits_per_day Int32,  
        group Int32,  
        PRIMARY KEY (id)
    ) ENGINE = MergeTree()
    ORDER BY id;
    '''
    
    # Выполняем создание таблицы
    CH_CLIENT.execute(sql_request)
    print('Таблица успешно создана')

    chunk_qty = math.ceil(df.shape[0] / chunk_size)

    for i in range(chunk_qty):

        try:
            start = i * chunk_size
            end = (i + 1) * chunk_size
            df_short = df[start:end]
            # Преобразуем DataFrame в список словарей для вставки в базу данных
            columns = ['id', 'datetime', 'user_id', 'action', 'type', 'first_visit_unique', 'visits_per_day', 'group']
            data_to_insert = df_short[columns].to_dict('records')
            
            # Вставляем данные в таблицу
            insert_query = f'INSERT INTO {table_name} (id, datetime, user_id, action, type, first_visit_unique, visits_per_day, group) VALUES'
            CH_CLIENT.execute(insert_query, data_to_insert)
            
            print(f'Данные {i} из {chunk_qty - 1} успешно вставлены в таблицу')
            time.sleep(5)
        except:
            print(f'Ошибка при вставке чанка {i}')
            try:
                time.sleep(5)
                start = i * chunk_size
                end = (i + 1) * chunk_size
                df_short = df[start:end]
                # Преобразуем DataFrame в список словарей для вставки в базу данных
                columns = ['id', 'datetime', 'user_id', 'action', 'type', 'first_visit_unique', 'visits_per_day', 'group']
                data_to_insert = df_short[columns].to_dict('records')
                
                # Вставляем данные в таблицу
                insert_query = f'INSERT INTO {table_name} (id, datetime, user_id, action, type, first_visit_unique, visits_per_day, group) VALUES'
                CH_CLIENT.execute(insert_query, data_to_insert)
                
                print(f'Данные {i} из {chunk_qty - 1} успешно вставлены в таблицу')
            except Exception as e:
                print(e)
                print(f'Ошибка при вставке чанка {i}')
                print(f'Данные строк от {start} до{end} не вставлены')
    
                
            

## Nonpermanent_users insert

In [5]:
%%time

df = pd.read_csv('../data_preparation/nonpermanent_users.csv')
df.head()

CPU times: user 677 ms, sys: 82.6 ms, total: 759 ms
Wall time: 772 ms


,datetime,user_id,action,type,first_visit_unique,visits_per_day,group
0,2025-01-01 00:00:00.000001,1,view,main,2025-01-01 11:45:06.374215,1.0,1
1,2025-01-02 00:00:00.000001,1,view,main,2025-01-01 11:45:06.374215,16.0,1
2,2025-01-03 00:00:00.000001,1,view,main,2025-01-01 11:45:06.374215,1.0,1
3,2025-01-04 00:00:00.000001,1,view,main,2025-01-01 11:45:06.374215,5.0,1
4,2025-01-05 00:00:00.000001,1,view,main,2025-01-01 11:45:06.374215,3.0,1


In [6]:
%%time
table_name = 'nonpermanent_users_daily'
insert_daily_data(df, table_name)

Таблица успешно создана
Данные 0 из 17 успешно вставлены в таблицу
Данные 1 из 17 успешно вставлены в таблицу
Данные 2 из 17 успешно вставлены в таблицу
Данные 3 из 17 успешно вставлены в таблицу
Данные 4 из 17 успешно вставлены в таблицу
Данные 5 из 17 успешно вставлены в таблицу
Данные 6 из 17 успешно вставлены в таблицу
Данные 7 из 17 успешно вставлены в таблицу
Данные 8 из 17 успешно вставлены в таблицу
Данные 9 из 17 успешно вставлены в таблицу
Данные 10 из 17 успешно вставлены в таблицу
Данные 11 из 17 успешно вставлены в таблицу
Данные 12 из 17 успешно вставлены в таблицу
Данные 13 из 17 успешно вставлены в таблицу
Данные 14 из 17 успешно вставлены в таблицу
Данные 15 из 17 успешно вставлены в таблицу
Данные 16 из 17 успешно вставлены в таблицу
Данные 17 из 17 успешно вставлены в таблицу
CPU times: user 21.9 s, sys: 766 ms, total: 22.7 s
Wall time: 1min 57s


## Permanent users insert

In [7]:
%%time
df = pd.read_csv('../data_preparation/permanent_users.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6217337 entries, 0 to 6217336
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   datetime            object
 1   user_id             int64 
 2   action              object
 3   type                object
 4   first_visit_unique  object
 5   visits_per_day      int64 
 6   group               int64 
dtypes: int64(3), object(4)
memory usage: 332.0+ MB
CPU times: user 2.62 s, sys: 273 ms, total: 2.9 s
Wall time: 2.98 s


In [8]:
%%time
table_name = 'permanent_users_daily'
insert_daily_data(df, table_name)

Таблица успешно создана
Данные 0 из 62 успешно вставлены в таблицу
Данные 1 из 62 успешно вставлены в таблицу
Данные 2 из 62 успешно вставлены в таблицу
Данные 3 из 62 успешно вставлены в таблицу
Данные 4 из 62 успешно вставлены в таблицу
Данные 5 из 62 успешно вставлены в таблицу
Данные 6 из 62 успешно вставлены в таблицу
Данные 7 из 62 успешно вставлены в таблицу
Данные 8 из 62 успешно вставлены в таблицу
Данные 9 из 62 успешно вставлены в таблицу
Данные 10 из 62 успешно вставлены в таблицу
Данные 11 из 62 успешно вставлены в таблицу
Данные 12 из 62 успешно вставлены в таблицу
Данные 13 из 62 успешно вставлены в таблицу
Данные 14 из 62 успешно вставлены в таблицу
Данные 15 из 62 успешно вставлены в таблицу
Данные 16 из 62 успешно вставлены в таблицу
Данные 17 из 62 успешно вставлены в таблицу
Данные 18 из 62 успешно вставлены в таблицу
Данные 19 из 62 успешно вставлены в таблицу
Данные 20 из 62 успешно вставлены в таблицу
Данные 21 из 62 успешно вставлены в таблицу
Данные 22 из 62 ус